## 3. Mapping, sorting, and calling trimmed data

Using [NextGenMap](https://github.com/Cibiv/NextGenMap) for mapping and [samtools](https://github.com/samtools/samtools) for sorting. Alternatives are [minimap2](https://github.com/lh3/minimap2) and [Sambamba](https://github.com/biod/sambamba). Even a quite powerful personal desktop may freeze up or not start running MiniMap2 due to its memory requirements. NextGenMap seems to play better but when dealing with a HPC cluster, either should work fine.

After this, we will use [FreeBayes](https://github.com/ekg/freebayes) for variant detection on the mapped and sorted files.

#### Mapping:

In [2]:
##conda install -c bioconda nextgenmap

In [ ]:
%time !ngm -r ./reference-genome/Mmul8.fna.gz -1 out.R1.fq.gz -2 out.R2.fq.gz -o results.sam -t 10 ##took ~140mins on my system. all time estimates are for my meager desktop on the smallest genome-fascicularis

In [13]:
##conda install -c bioconda samtools

#### SAM to BAM conversion and sorting reads:

In [6]:
%time !samtools view -S -b results.sam > sample.bam ##simple conversion to bam appx 21 min on a 12 thread desktop w/ 16gb ram, not bad

CPU times: user 23.8 s, sys: 3.48 s, total: 27.3 s
Wall time: 21min 13s


In [7]:
%time !samtools sort sample.bam -o sample.sorted.bam ##sorting bam file into genome order ~26mins

[bam_sort_core] merging from 53 files and 1 in-memory blocks...
CPU times: user 28.7 s, sys: 3.85 s, total: 32.6 s
Wall time: 25min 41s


In [14]:
%time !samtools index sample.sorted.bam ##of course this will all be piped together...

CPU times: user 2.36 s, sys: 363 ms, total: 2.73 s
Wall time: 1min 57s


In [1]:
%time !samtools view sample.sorted.bam | head -n 1 ##We see that instead of giving chromosomes logical names like Chr1, Chr2, etc., the reference genome has strange names for chromosomes (NC_027893.1, etc)...

SRR445694~125200.sra.858593	99	NC_027893.1	1	60	5S95M	=	294	393	AAGGCCATGGAAACAAGGAAAGTCTGAAAAACTCACAGTTTAGGAACCTAAAGAGACTTGACTACTAAATGGAATATATCTTGGGATCCTGGAAAAGAAA	CCCFFFFFHHHHHIIIIIIIIIHIIIIIIIIIIIIIIIHIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIHFHHHHFFFFFFDDDDCCCDBCCDDBDD	AS:i:950	NM:i:0	NH:i:0	XI:f:1	X0:i:0	XE:i:28	XR:i:95	MD:Z:95
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1
CPU times: user 5.74 ms, sys: 7.92 ms, total: 13.7 ms
Wall time: 224 ms


In [3]:
%time !samtools view -h -b sample.sorted.bam NC_027893.1 > chr1.bam ##Which makes splitting files up for chromosome-level analyses a bit more annoying but not too bad...I'll make a bash script

CPU times: user 1.19 s, sys: 147 ms, total: 1.34 s
Wall time: 58.6 s


Pipe from NGM to samtools with an output of a sorted bam file:

In [ ]:
!ngm -r ./reference-genome/Mmul8.fna.gz -1 out.R1.fq.gz -2 out.R2.fq.gz | samtools view -S -b | samtools sort -o sample.sorted.bam

#### Variant calling:

In [ ]:
!freebayes -f ./reference-genome/Mmul8.fna.gz sample.sorted.bam >wholegenome.vcf ##example code for variant calling on entire genome. Can be split by chromosome/region using -r 